# Setup:

This requires a reference .fa file and a .gtf file (here we are using the ones from Maestri, human genome)
IF the bash commands fail from the notebook, just use the terminal


## 1. Setting up the files for chromosome 1

In [25]:
%%bash
startloc="/globalscratch/ulb/mlg/hdmarmol/nanolympics/startingfiles/references"
endloc="/globalscratch/ulb/mlg/hdmarmol/nanolympics/startingfiles/chr1"


In [ ]:
# This checks for allthe lines whose first column is only equal tothe chromosome number we want (here 1)
%%bash
awk '$1 == "1"' "${startloc}/Homo_sapiens.GRCh38.109.gtf" > "${endloc}/chr1.gtf"


In [ ]:
# create a .fai file from the .fa file for the region of interest. If it doesn't work do it from terminal

%%bash
# Load the right release 
module load releases/2022b
module load SAMtools/1.17-GCC-12.2.0

samtools faidx "${startloc}/hg38.fa" chr1 > "${endloc}/chr1.fa"

In [ ]:
# Convert the GTF to BED format: This makes an EXON bed. They have the gene id

%%bash
awk '$3 == "exon" { match($9, /gene_id "[^"]+"/); gene_id = $10; print $1 "\t" $4 "\t" $5 "\t" gene_id "\t.\t" $7}' "${endloc}/chr1.gtf" > "${endloc}/chr1_exons.bed"

# IF there is a mismatch in the chromosome naming between the files:
awk 'BEGIN {OFS="\t"} {if ($1 == "1") $1 = "chr1"; print}' "${endloc}/chr1_exons.bed" > "${endloc}/chr1_exons_fixed.bed"

In [ ]:
# Converts the GTF to BED format: This makes a transcriptome bed, with transcript id
%%bash
awk '$3 == "transcript" {transcript_id = $14; gsub(/;$/, "", transcript_id); \
    gsub(/"/, "", transcript_id); print $1 "\t" $4 "\t" $5 "\t" transcript_id "\t.\t" $7;}' \
    "${endloc}/chr1.gtf" > "${endloc}/chr1_transcriptome.bed"
awk 'BEGIN {OFS="\t"} {if ($1 == "1") $1 = "chr1"; print}' "${endloc}/chr1_transcriptome.bed" > "${endloc}/chr1_transcriptome_fixed.bed"

# Converts the GTF to BED format: This makes a Gene bed

awk '$3 == "gene" {gene_id = $10; gsub(/;$/, "", gene_id); gsub(/"/, "", gene_id); print $1 "\t" $4 "\t" $5 "\t" gene_id "\t.\t" $7;}' "${endloc}/chr1.gtf" > "${endloc}/chr1_genes.bed"
awk 'BEGIN {OFS="\t"} {if ($1 == "1") $1 = "chr1"; print}' "${endloc}/chr1_genes.bed" > "${endloc}/chr1_genes_fixed.bed"



In [ ]:
# Make a transcriptome FASTA
%%bash
module load releases/2023a
module load BEDTools/2.31.0-GCC-12.3.0

bedtools getfasta -fi "${endloc}/chr1.fa" -bed "${endloc}/chr1_transcriptome_fixed.bed" -name -split > "${endloc}/chr1_transcriptome.fa"

In [ ]:
# Make the FAI for the transcriptome FASTA

%%bash
# Load the right release 
module load releases/2022b
module load SAMtools/1.17-GCC-12.2.0

samtools faidx "${endloc}/chr1_transcriptome.fa" > "${endloc}/chr1_transcriptome_other.fa"


## 2: Making the genestoranscript file

In [3]:
# Variables

gtfFile = "/globalscratch/ulb/mlg/hdmarmol/nanolympics/startingfiles/chr1/chr1.gtf"
genesToTanscript = "/globalscratch/ulb/mlg/hdmarmol/nanolympics/startingfiles/chr1/chr1_genesToTranscript.txt"
separator = "\t"

In [66]:
# making a dictionary of genes: [transcripts]
# Ideally I should make a condition to just find the column that has gene_name and take the one next to it, or even better split 
# by ";" and check for the one that startswith what I want

geneDict = {}
biotypes = []

try:
    with open(gtfFile, "r") as f:
            for line in f:
                #line = f.readline()
                line_ls = line.split("\t")
                #print (line_ls[0:8])
                info = line_ls[8].split(" ")
                #print(info)
                
                # Check if new gene
                if line_ls[2] == "transcript":
                    gene_name = info[9]
                    gene_name = gene_name[1:len(gene_name)-2]
                    transcript_id = info[5][1:len(info[5])-2]
                    #print(gene_name, transcript_id)
                    if info[8] != "gene_name":
                    #print(line)
                        #print(info)
                        if info[11] not in biotypes:
                            biotypes.append(info[11])
                        continue
                    if gene_name not in geneDict.keys():
                        geneDict[gene_name] = []
                    if transcript_id not in geneDict[gene_name]:
                        geneDict[gene_name].append(transcript_id)
            print(geneDict)
            print(geneDict.keys())
            print(len(geneDict.keys()))
            print(biotypes)
    
    with open(genesToTanscript, "w") as f:
        for (k,v) in geneDict.items():
            line = k
            for transcript in v:
                line += f"\t{transcript}"
            line += "\n"
            f.write(line)
            
except:
    print(".gtf File not found in path")
    



        


{'ATAD3B': ['ENST00000673477', 'ENST00000472194', 'ENST00000378736', 'ENST00000485748', 'ENST00000474481', 'ENST00000308647'], 'DDX11L17': ['ENST00000624431'], 'PRDM16': ['ENST00000511072', 'ENST00000607632', 'ENST00000378391', 'ENST00000514189', 'ENST00000270722', 'ENST00000512462', 'ENST00000463591', 'ENST00000509860', 'ENST00000378389', 'ENST00000606170'], 'PEX10': ['ENST00000288774', 'ENST00000447513', 'ENST00000650293', 'ENST00000507596', 'ENST00000510434', 'ENST00000508384', 'ENST00000515760', 'ENST00000502666', 'ENST00000514502'], 'LINC01345': ['ENST00000635247', 'ENST00000635127', 'ENST00000425194', 'ENST00000671015'], 'EEF1DP6': ['ENST00000449232'], 'PEX14': ['ENST00000472851', 'ENST00000356607', 'ENST00000491661', 'ENST00000492696'], 'LINC01646': ['ENST00000634256', 'ENST00000635556', 'ENST00000420522', 'ENST00000659059', 'ENST00000667354', 'ENST00000659083', 'ENST00000661628', 'ENST00000666685'], 'LINC01777': ['ENST00000635002', 'ENST00000423197', 'ENST00000635312', 'ENST000